In [61]:
from lib.ekyn import *
from torch.utils.data import DataLoader,ConcatDataset,TensorDataset
from lib.datasets import EpochedDataset
from sklearn.model_selection import train_test_split
import torch
from lib.models import RegNetX
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn
from torch.nn.functional import relu

CONFIG = {
    'BATCH_SIZE':256,
    'WINDOW_SIZE':5000,
    'LEARNING_RATE':3e-3,
    'TEST_SIZE':.25,
    'DEVICE':'cuda',
    'DEPTHI':[1,2,4,1],
    'WIDTHI':[8,16,32,64],
    'STEM_KERNEL_SIZE':3
}

train_idx,test_idx = train_test_split(get_ekyn_ids(),test_size=.25,random_state=0)
print(train_idx,test_idx)
trainloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in train_idx for condition in ['Vehicle','PF']]),batch_size=CONFIG['BATCH_SIZE'],shuffle=True)
devloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in test_idx for condition in ['Vehicle','PF']]),batch_size=CONFIG['BATCH_SIZE'],shuffle=True)

['F1-0', 'B3-1', 'A4-0', 'F1-1', 'E2-1', 'C4-1', 'F5-1', 'E4-0', 'B1-0', 'A1-0', 'C1-0', 'E4-1'] ['A1-1', 'C4-0', 'D1-0', 'E1-0']


In [62]:
from lib.models import YBlock
import math
class YBlock(nn.Module):
    def __init__(self,in_channels,out_channels,seq_len) -> None:
        super().__init__()
        stride = 1
        self.seq_len = seq_len
        self.in_channels = in_channels
        self.out_channels = out_channels
        if out_channels > in_channels:
            stride = 2
            self.seq_len = seq_len // 2 if seq_len % 2 == 0 else seq_len // 2 + 1
        print(self.seq_len)
        self.block = nn.Sequential(
            nn.Conv1d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=stride,padding=1,bias=False),
            nn.ReLU()
            
        )
        self.identity = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1, padding=0, stride=stride),
            nn.ReLU()
        ) if out_channels > in_channels else nn.Identity()
        self.relu = nn.ReLU()
    def forward(self,x):
        x = self.relu(self.block(x) + self.identity(x))
        return x
class RegNetY(nn.Module):
    def __init__(self,depth,width,stem_kernel_size) -> None:
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv1d(in_channels=1,out_channels=width[0],kernel_size=stem_kernel_size,stride=2,padding=stem_kernel_size//2,bias=False),
            nn.MaxPool1d(kernel_size=2,stride=2),
            nn.ReLU()
        )

        self.body = nn.Sequential()
        for stage_i in range(len(width)):
            print(f'stage {stage_i}, depth {depth[stage_i]}')
            for block_i in range(depth[stage_i]):
                self.body.add_module(name=f'{stage_i}_{block_i}',module=YBlock(in_channels=width[0] if stage_i == 0 and block_i == 0 else self.body[-1].out_channels,out_channels=width[stage_i],seq_len=self.body[-1].seq_len if stage_i > 0 else 1250))
                print(f'block {stage_i} {self.body[-1].in_channels} {self.body[-1].out_channels}')

        self.classifier = nn.Sequential(
            nn.AvgPool1d(kernel_size=self.body[-1].seq_len),
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=width[-1],out_features=3)
        )

    def forward(self,x):
        x = x.view(-1,1,5000)
        x = self.stem(x)
        x = self.body(x)
        x = self.classifier(x)
        return x
model = RegNetY(depth=CONFIG['DEPTHI'],width=CONFIG['WIDTHI'],stem_kernel_size=CONFIG['STEM_KERNEL_SIZE'])
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=CONFIG['LEARNING_RATE'])
# model.to(CONFIG['DEVICE']);

stage 0, depth 1
1250
block 0 8 8
stage 1, depth 2
625
block 1 8 16
625
block 1 16 16
stage 2, depth 4
313
block 2 16 32
313
block 2 32 32
313
block 2 32 32
313
block 2 32 32
stage 3, depth 1
157
block 3 32 64


In [63]:
Xi,yi = next(iter(trainloader))

In [64]:
model(Xi).shape

torch.Size([256, 3])

In [65]:
lossi = []
trainlossi = []
trainf1 = []
devlossi = []
devf1 = []
model.train()
from lib.utils import training_loop,development_loop
for i in tqdm(range(1000)):
    loss,f1 = training_loop(model=model,trainloader=trainloader,criterion=criterion,optimizer=optimizer,device=CONFIG['DEVICE'])
    trainlossi.append(loss)
    trainf1.append(f1)

    # if i % 100 == 0:
    #     loss,f1 = development_loop(model=model,devloader=devloader,criterion=criterion,device=CONFIG['DEVICE'])
    #     devlossi.append(loss)
    #     devf1.append(f1)

    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,4))
    ax[0].plot(trainlossi)
    ax[0].plot(torch.linspace(0,len(trainlossi),len(devlossi)),devlossi)
    ax[1].plot(trainf1)
    ax[1].plot(torch.linspace(0,len(trainf1),len(devf1)),devf1)
    plt.savefig('loss.jpg')
    plt.close()

  1%|          | 9/1000 [01:55<3:31:42, 12.82s/it]


KeyboardInterrupt: 

In [66]:
from lib.utils import evaluate
evaluate(devloader,model,criterion,CONFIG['DEVICE'])

100%|██████████| 270/270 [00:01<00:00, 169.00it/s]


(0.26000638328216696,
 {'precision': 0.8113643714116306,
  'recall': 0.8448450140676802,
  'f1': 0.8258637231317404},
 tensor([1., 1., 1.,  ..., 1., 1., 2.]),
 tensor([1., 1., 1.,  ..., 2., 1., 2.]),
 tensor([[3.6073e-06, 9.8673e-01, 1.3270e-02],
         [6.9646e-10, 9.9950e-01, 5.0115e-04],
         [1.8734e-06, 9.9287e-01, 7.1251e-03],
         ...,
         [4.0691e-02, 9.7651e-02, 8.6166e-01],
         [1.1035e-08, 9.9757e-01, 2.4299e-03],
         [8.5303e-04, 1.5289e-01, 8.4626e-01]]))